In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from keras.layers import Dense, Dropout
from keras.models import Model, Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import time

In [2]:
df = pd.read_csv("kuzma_dataset.csv",index_col=0)
df.head()

,G,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,...,BLK,TOV,PF,PTS,GmSc,"""+/-""",Win,Winby,minutes,age_y
0,1,4,8,0.500,0,2,0.000,0,2,0.000,...,0,0,2,8,7.1,-17,0,-16,18.700000,22.238356
1,2,6,7,0.857,1,2,0.500,2,3,0.667,...,0,0,0,15,14.4,-18,1,2,28.400000,22.241096
2,3,7,13,0.538,2,6,0.333,4,4,1.000,...,0,2,3,20,15.1,2,0,-7,30.433333,22.246575
3,4,6,15,0.400,1,6,0.167,2,2,1.000,...,0,3,1,15,4.4,4,1,3,33.783333,22.254795
4,5,7,14,0.500,1,5,0.200,0,0,NaN,...,0,4,2,15,7.7,3,0,-9,32.000000,22.260274


In [3]:
df = df.astype('float32')

In [4]:
y = df['Win']
X = df.drop(columns=['Win','Winby'])

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=y, random_state=42)

In [6]:
def my_DL(epochs=6,batchsize=512):
    model = Sequential()
    model.add(Dense(32,activation='relu'))
    model.add(Dense(16,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='rmsprop', metrics=['accuracy'])
    return model

In [7]:
def train_hyper_tune(X,y):
    # create the pre-processing component
    my_scaler = StandardScaler()
    my_imputer = SimpleImputer(strategy="median")
    
    # define classifiers
    ## Classifier 1: Logistic Regression
    clf_LR = LogisticRegression(random_state=0,penalty='elasticnet',solver='saga')
    ## Classifier 2: Random Forest Classifier
    clf_RF = RandomForestClassifier(random_state=0)
    ## Classifier 3: Deep Learning Binary Classifier
    clf_DL = KerasClassifier(build_fn=my_DL)
    
    # define pipeline for three classifiers
    ## clf_LR
    pipe1 = Pipeline([('imputer', my_imputer), ('scaler', my_scaler), ('lr_model',clf_LR)])
    ## clf_RF
    pipe2 = Pipeline([('imputer', my_imputer), ('scaler', my_scaler), ('rf_model',clf_RF)])
    ## clf_DL
    pipe3 = Pipeline([('imputer', my_imputer), ('scaler', my_scaler), ('dl_model',clf_DL)])
    
    # create hyperparameter space of the three models
    ## clf_LR
    param_grid1 = {
        'lr_model__C' : [1e-1,1,10],
        'lr_model__l1_ratio' : [0,0.5,1]
    }
    ## clf_RF
    param_grid2 = {
        'rf_model__n_estimators' : [50,100],
        'rf_model__max_features' : [0.8,"auto"],
        'rf_model__max_depth' : [4,5]
    }
    ## clf_DL
    param_grid3 = {
        'dl_model__epochs' : [6,12,18,24],
        'dl_model__batchsize' : [256,512]
    }
    
    # set GridSearch via 5-fold cross-validation
    ## clf_LR
    grid1 = GridSearchCV(pipe1, cv=5, param_grid=param_grid1)
    ## clf_RF
    grid2 = GridSearchCV(pipe2, cv=5, param_grid=param_grid2)
    ## clf_DL
    grid3 = GridSearchCV(pipe3, cv=5, param_grid=param_grid3)
    
    # run the hyperparameter tunning process
    start_LR = time.perf_counter()
    grid1.fit(X,y)
    finish_LR = time.perf_counter()
    start_RF = time.perf_counter()
    grid2.fit(X,y)
    finish_RF = time.perf_counter()
    start_DL = time.perf_counter()
    grid3.fit(X,y)
    finish_DL = time.perf_counter()
    print("Training Time For LR: ",finish_LR-start_LR, "seconds")
    print("Training Time For RF: ",finish_RF-start_RF, "seconds")
    print("Training Time For DL: ",finish_DL-start_DL, "seconds")
    # return results of the tunning process
    return grid1,grid2,grid3,pipe1,pipe2,pipe3

In [8]:
my_grid1,my_grid2,my_grid3,my_pipe1,my_pipe2,my_pipe3 = train_hyper_tune(X_train, y_train)

/Users/rushipatel/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/rushipatel/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/rushipatel/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/rushipatel/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/rushipatel/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sa

Epoch 1/6
5/5 [==============================] - 0s 1ms/step - loss: 0.6615 - accuracy: 0.5205
Epoch 2/6
5/5 [==============================] - 0s 1ms/step - loss: 0.6042 - accuracy: 0.7055
Epoch 3/6
5/5 [==============================] - 0s 3ms/step - loss: 0.5731 - accuracy: 0.7329
Epoch 4/6
5/5 [==============================] - 0s 4ms/step - loss: 0.5512 - accuracy: 0.7534
Epoch 5/6
5/5 [==============================] - 0s 3ms/step - loss: 0.5308 - accuracy: 0.7603
Epoch 6/6
2/2 [==============================] - 0s 964us/step - loss: 0.6135 - accuracy: 0.5946
Epoch 1/6
5/5 [==============================] - 0s 974us/step - loss: 0.7663 - accuracy: 0.3973
Epoch 2/6
5/5 [==============================] - 0s 1ms/step - loss: 0.7138 - accuracy: 0.4384
Epoch 3/6
5/5 [==============================] - 0s 3ms/step - loss: 0.6857 - accuracy: 0.5274
Epoch 4/6
5/5 [==============================] - 0s 2ms/step - loss: 0.6637 - accuracy: 0.5890
Epoch 5/6
5/5 [==============================]

5/5 [==============================] - 0s 1ms/step - loss: 0.6350 - accuracy: 0.6644
Epoch 3/12
5/5 [==============================] - 0s 2ms/step - loss: 0.6108 - accuracy: 0.6918
Epoch 4/12
5/5 [==============================] - 0s 2ms/step - loss: 0.5938 - accuracy: 0.7192
Epoch 5/12
5/5 [==============================] - 0s 2ms/step - loss: 0.5761 - accuracy: 0.7055
Epoch 6/12
5/5 [==============================] - 0s 2ms/step - loss: 0.5623 - accuracy: 0.7466
Epoch 7/12
5/5 [==============================] - 0s 3ms/step - loss: 0.5489 - accuracy: 0.7534
Epoch 8/12
5/5 [==============================] - 0s 2ms/step - loss: 0.5352 - accuracy: 0.7534
Epoch 9/12
5/5 [==============================] - 0s 3ms/step - loss: 0.5238 - accuracy: 0.7671
Epoch 10/12
5/5 [==============================] - 0s 2ms/step - loss: 0.5115 - accuracy: 0.7877
Epoch 11/12
5/5 [==============================] - 0s 3ms/step - loss: 0.5008 - accuracy: 0.7877
Epoch 12/12
2/2 [==============================] 

2/2 [==============================] - 0s 921us/step - loss: 0.5991 - accuracy: 0.5676
Epoch 1/18
5/5 [==============================] - 0s 2ms/step - loss: 0.6948 - accuracy: 0.5548
Epoch 2/18
5/5 [==============================] - 0s 2ms/step - loss: 0.6522 - accuracy: 0.6164
Epoch 3/18
5/5 [==============================] - 0s 3ms/step - loss: 0.6267 - accuracy: 0.6918
Epoch 4/18
5/5 [==============================] - 0s 2ms/step - loss: 0.6062 - accuracy: 0.7192
Epoch 5/18
5/5 [==============================] - 0s 3ms/step - loss: 0.5862 - accuracy: 0.7329
Epoch 6/18
5/5 [==============================] - 0s 2ms/step - loss: 0.5690 - accuracy: 0.7260
Epoch 7/18
5/5 [==============================] - 0s 2ms/step - loss: 0.5522 - accuracy: 0.7192
Epoch 8/18
5/5 [==============================] - 0s 1ms/step - loss: 0.5356 - accuracy: 0.7740
Epoch 9/18
5/5 [==============================] - 0s 1ms/step - loss: 0.5206 - accuracy: 0.8014
Epoch 10/18
5/5 [==============================] 

5/5 [==============================] - 0s 1ms/step - loss: 0.6612 - accuracy: 0.6327
Epoch 5/18
5/5 [==============================] - 0s 2ms/step - loss: 0.6400 - accuracy: 0.6463
Epoch 6/18
5/5 [==============================] - 0s 3ms/step - loss: 0.6203 - accuracy: 0.6531
Epoch 7/18
5/5 [==============================] - 0s 3ms/step - loss: 0.6001 - accuracy: 0.6871
Epoch 8/18
5/5 [==============================] - 0s 3ms/step - loss: 0.5826 - accuracy: 0.6871
Epoch 9/18
5/5 [==============================] - 0s 2ms/step - loss: 0.5680 - accuracy: 0.7143
Epoch 10/18
5/5 [==============================] - 0s 2ms/step - loss: 0.5528 - accuracy: 0.7415
Epoch 11/18
5/5 [==============================] - 0s 3ms/step - loss: 0.5359 - accuracy: 0.7551
Epoch 12/18
5/5 [==============================] - 0s 2ms/step - loss: 0.5223 - accuracy: 0.7483
Epoch 13/18
5/5 [==============================] - 0s 2ms/step - loss: 0.5074 - accuracy: 0.7755
Epoch 14/18
5/5 [==============================

...

5/5 [==============================] - 0s 1ms/step - loss: 0.6551 - accuracy: 0.6463
Epoch 3/12
5/5 [==============================] - 0s 2ms/step - loss: 0.6312 - accuracy: 0.6735
Epoch 4/12
5/5 [==============================] - 0s 2ms/step - loss: 0.6141 - accuracy: 0.7007
Epoch 5/12
5/5 [==============================] - 0s 2ms/step - loss: 0.5988 - accuracy: 0.7075
Epoch 6/12
5/5 [==============================] - 0s 3ms/step - loss: 0.5855 - accuracy: 0.7211
Epoch 7/12
5/5 [==============================] - 0s 2ms/step - loss: 0.5734 - accuracy: 0.7211
Epoch 8/12
5/5 [==============================] - 0s 2ms/step - loss: 0.5607 - accuracy: 0.7279
Epoch 9/12
5/5 [==============================] - 0s 2ms/step - loss: 0.5502 - accuracy: 0.7619
Epoch 10/12
5/5 [==============================] - 0s 2ms/step - loss: 0.5396 - accuracy: 0.7687
Epoch 11/12
5/5 [==============================] - 0s 2ms/step - loss: 0.5304 - accuracy: 0.7755
Epoch 12/12
2/2 [==============================] 

Epoch 13/18
5/5 [==============================] - 0s 3ms/step - loss: 0.5332 - accuracy: 0.7671
Epoch 14/18
5/5 [==============================] - 0s 3ms/step - loss: 0.5202 - accuracy: 0.7945
Epoch 15/18
5/5 [==============================] - 0s 1ms/step - loss: 0.5073 - accuracy: 0.7877
Epoch 16/18
5/5 [==============================] - 0s 4ms/step - loss: 0.4951 - accuracy: 0.8014
Epoch 17/18
5/5 [==============================] - 0s 3ms/step - loss: 0.4838 - accuracy: 0.8151
Epoch 18/18
2/2 [==============================] - 0s 998us/step - loss: 0.5792 - accuracy: 0.6757
Epoch 1/18
5/5 [==============================] - 0s 1ms/step - loss: 0.7214 - accuracy: 0.4830
Epoch 2/18
5/5 [==============================] - 0s 3ms/step - loss: 0.6582 - accuracy: 0.5850
Epoch 3/18
5/5 [==============================] - 0s 4ms/step - loss: 0.6284 - accuracy: 0.6531
Epoch 4/18
5/5 [==============================] - 0s 2ms/step - loss: 0.6030 - accuracy: 0.6871
Epoch 5/18
5/5 [================

5/5 [==============================] - 0s 4ms/step - loss: 0.4132 - accuracy: 0.8219
Epoch 18/24
5/5 [==============================] - 0s 3ms/step - loss: 0.4010 - accuracy: 0.8151
Epoch 19/24
5/5 [==============================] - 0s 5ms/step - loss: 0.3912 - accuracy: 0.8288
Epoch 20/24
5/5 [==============================] - 0s 3ms/step - loss: 0.3806 - accuracy: 0.8493
Epoch 21/24
5/5 [==============================] - 0s 2ms/step - loss: 0.3701 - accuracy: 0.8630
Epoch 22/24
5/5 [==============================] - 0s 2ms/step - loss: 0.3602 - accuracy: 0.8630
Epoch 23/24
5/5 [==============================] - 0s 3ms/step - loss: 0.3511 - accuracy: 0.8493
Epoch 24/24
2/2 [==============================] - 0s 1ms/step - loss: 0.5960 - accuracy: 0.6757
Epoch 1/24
5/5 [==============================] - 0s 2ms/step - loss: 0.7098 - accuracy: 0.4521
Epoch 2/24
5/5 [==============================] - 0s 3ms/step - loss: 0.6635 - accuracy: 0.5479
Epoch 3/24
5/5 [============================

5/5 [==============================] - 0s 5ms/step - loss: 0.6231 - accuracy: 0.6122
Epoch 4/24
5/5 [==============================] - 0s 4ms/step - loss: 0.6062 - accuracy: 0.6463
Epoch 5/24
5/5 [==============================] - 0s 3ms/step - loss: 0.5920 - accuracy: 0.6803
Epoch 6/24
5/5 [==============================] - 0s 4ms/step - loss: 0.5782 - accuracy: 0.7007
Epoch 7/24
5/5 [==============================] - 0s 2ms/step - loss: 0.5658 - accuracy: 0.7075
Epoch 8/24
5/5 [==============================] - 0s 2ms/step - loss: 0.5548 - accuracy: 0.7075
Epoch 9/24
5/5 [==============================] - 0s 2ms/step - loss: 0.5428 - accuracy: 0.7279
Epoch 10/24
5/5 [==============================] - 0s 3ms/step - loss: 0.5330 - accuracy: 0.7415
Epoch 11/24
5/5 [==============================] - 0s 3ms/step - loss: 0.5224 - accuracy: 0.7415
Epoch 12/24
5/5 [==============================] - 0s 2ms/step - loss: 0.5111 - accuracy: 0.7823
Epoch 13/24
5/5 [==============================]

6/6 [==============================] - 0s 2ms/step - loss: 0.3909 - accuracy: 0.8251
Epoch 23/24
6/6 [==============================] - 0s 2ms/step - loss: 0.3837 - accuracy: 0.8251
Epoch 24/24
6/6 [==============================] - 0s 2ms/step - loss: 0.3766 - accuracy: 0.8306
Training Time For LR:  0.7488071810000179 seconds
Training Time For RF:  5.588240763999977 seconds
Training Time For DL:  42.13921638100004 seconds


In [9]:
def train_on_entire(X,y,pipe,grid_res):
    # fit pipeline
    pipe.set_params(**grid_res.best_params_).fit(X, y)
    # return the newly trained pipeline
    return pipe

In [10]:
start = time.perf_counter()
print("Accuracy for LR: ",train_on_entire(X_train,y_train,my_pipe1,my_grid1).score(X_test,y_test))
finish = time.perf_counter()
print("Execution Time for LR: ",finish-start, "seconds")

Accuracy for LR:  0.8260869565217391
Execution Time for LR:  0.02137896700003239 seconds


In [11]:
start = time.perf_counter()
print("Accuracy for RF: ",train_on_entire(X_train,y_train,my_pipe2,my_grid2).score(X_test,y_test))
finish = time.perf_counter()
print("Execution Time for RF: ",finish-start, "seconds")

Accuracy for RF:  0.782608695652174
Execution Time for RF:  0.20814885800001548 seconds


In [12]:
start = time.perf_counter()
print("Accuracy for DL: ",train_on_entire(X_train,y_train,my_pipe3,my_grid3).score(X_test,y_test))
finish = time.perf_counter()
print("Execution Time for DL: ",finish-start, "seconds")

Epoch 1/24
6/6 [==============================] - 0s 3ms/step - loss: 0.7133 - accuracy: 0.4809
Epoch 2/24
6/6 [==============================] - 0s 8ms/step - loss: 0.6552 - accuracy: 0.6448
Epoch 3/24
6/6 [==============================] - 0s 4ms/step - loss: 0.6237 - accuracy: 0.6612
Epoch 4/24
6/6 [==============================] - 0s 2ms/step - loss: 0.5959 - accuracy: 0.7049
Epoch 5/24
6/6 [==============================] - 0s 3ms/step - loss: 0.5738 - accuracy: 0.7158
Epoch 6/24
6/6 [==============================] - 0s 2ms/step - loss: 0.5535 - accuracy: 0.7377
Epoch 7/24
6/6 [==============================] - 0s 3ms/step - loss: 0.5339 - accuracy: 0.7486
Epoch 8/24
6/6 [==============================] - 0s 2ms/step - loss: 0.5162 - accuracy: 0.7705
Epoch 9/24
6/6 [==============================] - 0s 4ms/step - loss: 0.4988 - accuracy: 0.7814
Epoch 10/24
6/6 [==============================] - 0s 2ms/step - loss: 0.4826 - accuracy: 0.8087
Epoch 11/24
6/6 [======================